In [1]:
import sounddevice as sd
import librosa
import numpy as np
import tensorflow as tf

In [2]:
# Load the trained model
MODEL_PATH = "model.h5"
model = tf.keras.models.load_model(MODEL_PATH)


c:\Users\danie\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\danie\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [3]:
# Load label encoder to decode predictions
import pickle
with open('label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

c:\Users\danie\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.4.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
# Record audio
def record_audio(duration=3, sample_rate=22050):
    print(f"Recording for {duration} seconds...")
    audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='float32')
    sd.wait()  # Wait for the recording to finish
    print("Recording complete.")
    return audio.flatten(), sample_rate

# Extract features
def extract_features_from_audio(audio, sample_rate):
    try:
        features = np.mean(librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=50).T, axis=0)
        return features
    except Exception as e:
        print(f"Error in feature extraction: {e}")
        return None

# Predict audio class
def predict_audio_class(audio_features):
    if audio_features is not None:
        audio_features = audio_features.reshape(1, -1, 1)  # Reshape for Conv1D
        prediction = model.predict(audio_features)
        predicted_class = label_encoder.inverse_transform([np.argmax(prediction)])
        return predicted_class[0]
    return "Unknown"

In [20]:
duration = 3  # Recording duration in seconds
audio, sample_rate = record_audio(duration=duration)

from IPython.display import Audio
Audio(audio, rate=sample_rate)

Recording for 3 seconds...
Recording complete.


In [21]:
features = extract_features_from_audio(audio, sample_rate)
predicted_class = predict_audio_class(features)
print(f"Predicted class: {predicted_class}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Predicted class: dog_bark


In [6]:
# Load the sound file
gun_audio, gun_sample_rate = librosa.load('clean-gun.mp3', sr=22050)

# Extract features from the loaded sound file
gun_features = extract_features_from_audio(gun_audio, gun_sample_rate)

# Predict the class of the loaded sound file
gun_predicted_class = predict_audio_class(gun_features)
print(f"Predicted class for gun sound: {gun_predicted_class}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Predicted class for gun sound: drilling
